In [122]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from nltk.corpus import stopwords
import re

# Ajustes de exibição
pd.set_option('display.max_columns', None)

In [ ]:
# utilizar nível da vaga
# Adicionar etapa de limpeza na coluna título da vaga (remover traços, barras, etc)

In [2]:
# Lista de funções
def print_all(lista):

    print([*lista])

def encontrar_trechos(df: pd.core.frame.DataFrame,coluna: str, texto: str, usa_regex: bool = False):
    return df[coluna][df[coluna].str.contains(texto,regex = usa_regex, case=False).fillna(False)]

def encontrar_logica(df,coluna,texto,usa_regex = False):
    return list(df[coluna].str.contains(texto,regex = usa_regex, case=False).fillna(False))

def encontrar_chaves(serie, texto,usa_regex = False):
    return serie[serie.index.str.contains(texto, regex = usa_regex, case= False)]

def palavras_associadas(dados, coluna, regex):
    observado_completo = ' '.join(encontrar_trechos(dados,coluna,regex,usa_regex=True).values)
    lista = [word for word in observado_completo.split()]
    contagem = Counter(lista).most_common()
    serie = pd.Series(dict(contagem))
    return contagem, serie

In [3]:
# Carregar os arquivos Parquet
applicants = pd.read_parquet("applicants.parquet")
prospects = pd.read_parquet("prospects.parquet")
vagas = pd.read_parquet("vagas.parquet")

In [4]:
# Renomear colunas para evitar conflitos e identificar origem
applicants = applicants.add_prefix("applicants_")
prospects = prospects.add_prefix("prospects_")
vagas = vagas.add_prefix("vagas_")

In [ ]:
def data_unify(applicants, prospects, vagas):
    
    
    applicants = applicants.add_prefix("applicants_")
    prospects = prospects.add_prefix("prospects_")
    vagas = vagas.add_prefix("vagas_")
    
    united_data = pd.merge(
        prospects,
        vagas,
        left_on="prospects_vaga_id",
        right_on="vagas_id_vaga",
        how="inner"
    )
    
    united_data = pd.merge(
        united_data,
        applicants,
        left_on="prospects_codigo",
        right_on="applicants_infos_basicas_codigo_profissional",
        how="inner"
    )
    
    united_data[united_data['prospects_situacao_candidado'].isin([
        "Não Aprovado pelo Cliente",
        "Contratado pela Decision",
        "Não Aprovado pelo RH",
        "Não Aprovado pelo Requisitante",
        "Sem interesse nesta vaga",
        "Contratado como Hunting",
        "Aprovado",
        "Recusado",
        "Proposta Aceita"
        ])
    ]
    
    return united_data


class vagas_features_engineering:
    
    def __init__(self, df: pd.DataFrame):
        """
        Inicializa a classe com um DataFrame.

        :param df: DataFrame contendo os dados das vagas.
        """
        self.df = df.copy()
        
    def titulo_vaga(self) -> None:
        """Descrição aqui"""
        
        mapa_palavras_chave_titulo = {
            'key_SAP': r'SAP',
            'key_SD': r'\bSD\b',
            'key_MM': r'\bMM\b',
            'key_ABAP': r'\bABAP\b',
            'key_AMS': r'AMS',
            'key_JAVA': r'java',
            'key_ORACLE': r'oracle',
            'key_CLOUD': r'cloud|AWS|Azure',
            'key_EBS': r'\bEBS\b',
            'key_DBA': r'\bDBA\b',
            'key_PROXXI': r'\bPROXXI\b',
            'key_C_HASH': r'C#',
            'key_OPERATIONS': r'Operations|Operações',
            'key_PMO': r'\bPMO\b|projetos',
            'key_ADM': r'\b(Adm|Administ|Administrativo|Administrador)\b',
            'key_MARKETING': r'market',
            'key_TI': r'\bTI\b',
            'key_SALESFORCE': r'Salesforce',
            'key_PROJETOS': r'Projetos',
            'key_DADOS': r'Dados|Data',
            'key_SECURITY': r'\b(Segurança|Security|Cyber)\b',
            'key_WEB': r'\bWeb\b',
            'key_SCRUM': r'\bScrum\b',
            'key_SERVICE': r'Service',
            'key_REACT_NATIVE_ANGULAR': r'\b(react|native|angular)',
            'key_NET': r'.Net',
            'key_Analista': r'Analista|Analyst',
            'key_DEVOPS': r'devops',
            'key_PYTHON': r'\bpython\b',
            'key_C_PLUS_PLUS': r'C\+\+',
            'key_COBOL': r'Cobol',
            'key_ANDROID': r'Android',
            'key_SQL': r'SQL',
            'key_fiscal': r'fiscal|contábil|contabilidade',
            'key_software': r'Software|Front|FullStack|back'
        }
        
        
        # Criação de colunas binárias que informam palavras chave ou não
        for coluna, regex_pattern in mapa_palavras_chave_titulo.items():
            self.df[coluna] = self.df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)

    @staticmethod
    def map_nivel(titulo: str)-> str:
        niveis_dict = {
            "Estágio/Trainee": r"estagi|trainee",
            "Assistente": r"assistente",
            "Júnior": r"junior|jr",
            "Pleno": r"pleno|pl",
            "Sênior": r"senior|sr|sênior|sn",
            "Coordenação": r"coordenador|líder",
            "Gerência": r"gerente|manager",
            "Diretoria": r"diretor"
        }
        
        for nivel, padrao in niveis_dict.items():
            if re.search(padrao, titulo):
                return nivel
            
        return "Outros"
    
    def nivel_vaga(self) -> None:
        """Descrição aqui"""
        
        mapeamento_numerico = {
            "Outros": 0,
            "Estágio/Trainee": 0,
            "Assistente": 1,
            "Júnior": 2,
            "Pleno": 3,
            "Sênior": 4,
            "Coordenação": 5,
            "Gerência": 6,
            "Diretoria": 7
        }
        
        self.df['nivel_vaga'] = self.df['titulo_vaga'].apply(self.map_nivel)
        self.df['nivel_vaga'] = self.df['nivel_vaga'].map(mapeamento_numerico)
        
    def tipo_contratacao(self) -> None:
        """Descrição aqui"""
        
        mapa_palavras_chave_tipo = {
            'tipo_escolha_do_candidato': r'candidato poderá escolher',
            'tipo_CLT': r'clt',
            'tipo_PJ': r'pj',
            'tipo_cooperado': r'cooperado',
            'tipo_estagio': r'estagiário',
            'tipo_hunting': r'hunting',
            'tipo_misto': r'^(?=.*\bclt\b)(?=.*\bpj\b)(?=.*\bcooperado\b)(?=.*\bestagiário\b)(?=.*\bhunting\b).*',
        }
        
        # Criação de colunas binárias que informam palavras chave ou não
        for coluna, regex_pattern in mapa_palavras_chave_tipo.items():
            self.df[coluna] = self.df['tipo_contratacao'].str.contains(regex_pattern, regex= True, case= False).astype(int)
        
        
    def linguas_vaga(self) -> None:
        """Descrição aqui"""
        
        mapa_linguas = {
            'Nenhum': 0,
            'Básico': 1,
            'Intermediário': 2,
            'Avançado': 3,
            'Técnico': 3,
            'Fluente': 4
        }
        
        # Aplicar o mapeamento em novas colunas
        self.df['score_ingles'] = self.df['nivel_ingles'].map(mapa_linguas)
        self.df['score_espanhol'] = self.df['nivel_espanhol'].map(mapa_linguas)
        
        

In [2]:
import pandas as pd
import numpy as np

# Definição do número de linhas
num_rows = 100

# Criação do DataFrame com colunas numéricas de escalas diferentes
df = pd.DataFrame({
    'Milhões': np.random.randint(1_000_000, 10_000_000, num_rows),  # Números na casa dos milhões
    'Milhares': np.random.randint(1_000, 10_000, num_rows),  # Números na casa dos milhares
    'Unidades': np.random.randint(1, 100, num_rows),  # Números menores
    'Decimais': np.random.uniform(0.1, 1.0, num_rows),  # Números decimais pequenos
    'Pequenos_Decimais': np.random.uniform(0.0001, 0.001, num_rows)  # Decimais muito pequenos
})

# Exibição do DataFrame
print(df.head())

   Milhões  Milhares  Unidades  Decimais  Pequenos_Decimais
0  3399592      3466        17  0.373684           0.000325
1  7270002      2136        98  0.818355           0.000802
2  6782561      3394        39  0.962911           0.000839
3  7907283      5614        43  0.756021           0.000929
4  2568447      3536        47  0.681562           0.000486


In [7]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler

def normalize_dataframe(df, method='minmax'):

    # Escolhe o scaler com base no método
    if method == 'minmax':
        scaler = MinMaxScaler()
    elif method == 'standard':
        scaler = StandardScaler()
    elif method == 'robust':
        scaler = RobustScaler()
    elif method == 'maxabs':
        scaler = MaxAbsScaler()
    else:
        raise ValueError("Método inválido. Escolha entre: 'minmax', 'standard', 'robust', 'maxabs'.")

    return pd.DataFrame(scaler.fit_transform(df), columns= df.columns)

In [10]:
normalize_dataframe(df, method='maxabs')

,Milhões,Milhares,Unidades,Decimais,Pequenos_Decimais
0,0.342905,0.348517,0.171717,0.376427,0.325650
1,0.733299,0.214781,0.989899,0.824363,0.804173
2,0.684132,0.341277,0.393939,0.969981,0.841601
3,0.797579,0.564505,0.434343,0.761572,0.931746
4,0.259070,0.355556,0.474747,0.686566,0.487310
...,...,...,...,...,...
95,0.797553,0.676923,0.303030,0.905865,0.300912
96,0.903819,0.764304,0.545455,0.675737,0.308775
97,0.785832,0.565611,0.989899,0.951503,0.208591
98,0.521724,0.690196,0.747475,0.560795,0.226006


In [ ]:
# Função de nível da vaga:
## Obs: Tenho que adicionar limpeza da coluna de título para minúsculo

# Função de tipo de contratação
## Obs: Tenho que adicionar limpeza da coluna de título para minúsculos

# Definir uso da coluna de prazo de contratação

# Criar coluna de inglês

# Criar coluna de espanhol
# Preciso limpar o texto

# Criar função que normalize todas as colunas (Min/Max scaler, Standart Scaler, etc)

In [26]:
# Primeiro merge: prospects + vagas
merged = pd.merge(
    prospects,
    vagas,
    left_on="prospects_vaga_id",
    right_on="vagas_id_vaga",
    how="inner"
)

# Segundo merge: resultado + applicants
final_df = pd.merge(
    merged,
    applicants,
    left_on="prospects_codigo",
    right_on="applicants_infos_basicas_codigo_profissional",
    how="inner"
)

In [27]:
# Filtra casos de MATCHES e NÃO MATCHES com base em valores selecionados
final_df = final_df[final_df['prospects_situacao_candidado'].isin(["Não Aprovado pelo Cliente",
                                                                   "Contratado pela Decision",
                                                                   "Não Aprovado pelo RH",
                                                                   "Não Aprovado pelo Requisitante",
                                                                   "Sem interesse nesta vaga",
                                                                   "Contratado como Hunting",
                                                                   "Aprovado",
                                                                   "Recusado",
                                                                   "Proposta Aceita"])]

final_df

,prospects_vaga_id,prospects_titulo,prospects_modalidade,prospects_quantidade_prospects,prospects_nome,prospects_codigo,prospects_situacao_candidado,prospects_data_candidatura,prospects_ultima_atualizacao,prospects_comentario,prospects_recrutador,vagas_id_vaga,vagas_data_requicisao,vagas_limite_esperado_para_contratacao,vagas_titulo_vaga,vagas_vaga_sap,vagas_cliente,vagas_solicitante_cliente,vagas_empresa_divisao,vagas_requisitante,vagas_analista_responsavel,vagas_tipo_contratacao,vagas_prazo_contratacao,vagas_objetivo_vaga,vagas_prioridade_vaga,vagas_origem_vaga,vagas_superior_imediato,vagas_nome,vagas_telefone,vagas_pais,vagas_estado,vagas_cidade,vagas_bairro,vagas_regiao,vagas_local_trabalho,vagas_vaga_especifica_para_pcd,vagas_faixa_etaria,vagas_horario_trabalho,vagas_nivel profissional,vagas_nivel_academico,vagas_nivel_ingles,vagas_nivel_espanhol,vagas_outro_idioma,vagas_areas_atuacao,vagas_principais_atividades,vagas_competencia_tecnicas_e_comportamentais,vagas_demais_observacoes,vagas_viagens_requeridas,vagas_equipamentos_necessarios,vagas_valor_venda,vagas_valor_compra_1,vagas_valor_compra_2,vagas_data_inicial,vagas_data_final,vagas_habilidades_comportamentais_necessarias,vagas_nome_substituto,vagas_categoria_contratacao,applicants_id_applicants,applicants_telefone_recado,applicants_telefone,applicants_infos_basicas_objetivo_profissional,applicants_infos_basicas_data_criacao,applicants_infos_basicas_inserido_por,applicants_infos_basicas_email,applicants_infos_basicas_local,applicants_infos_basicas_sabendo_de_nos_por,applicants_infos_basicas_data_atualizacao,applicants_infos_basicas_codigo_profissional,applicants_infos_basicas_nome,applicants_data_aceite,applicants_nome,applicants_cpf,applicants_fonte_indicacao,applicants_email,applicants_email_secundario,applicants_data_nascimento,applicants_telefone_celular,applicants_sexo,applicants_estado_civil,applicants_pcd,applicants_endereco,applicants_skype,applicants_url_linkedin,applicants_facebook,applicants_titulo_profissional,applicants_area_atuacao,applicants_conhecimentos_tecnicos,applicants_certificacoes,applicants_outras_certificacoes,applicants_remuneracao,applicants_nivel_profissional,applicants_nivel_academico,applicants_nivel_ingles,applicants_nivel_espanhol,applicants_outro_idioma,applicants_instituicao_ensino_superior,applicants_cursos,applicants_ano_conclusao,applicants_download_cv,applicants_qualificacoes,applicants_experiencias,applicants_outro_curso,applicants_id_ibrati,applicants_email_corporativo,applicants_cargo_atual,applicants_projeto_atual,applicants_cliente,applicants_unidade,applicants_data_admissao,applicants_data_ultima_promocao,applicants_nome_superior_imediato,applicants_email_superior_imediato,applicants_cv_pt,applicants_cv_en
2,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,2,Sra. Yasmin Fernandes,25364,Contratado pela Decision,2021-03-17,2021-04-12,Data de Inicio: 12/04/2021,Juliana Cassiano,4531,2021-03-10,NaT,2021-2607395-PeopleSoft Application Engine-Dom...,Não,Gonzalez and Sons,Valentim Duarte,Decision São Paulo,Vitória Melo,Srta. Bella Ferreira,PJ/Autônomo,,Contratação,Média: Média complexidade 6 a 10 dias,Nova Posição,Superior Imediato:,,,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Médio Completo,Nenhum,Nenhum,,Gestão e Alocação de Recursos de TI-,Key skills required for the job are:\n\nPeople...,O recurso Peoplesoft tem como responsabilidade...,"Remoto DEPOIS PRESENCIAL, TEMPO INDETERMINADO",,Notebook padrão -,0.0,hora,,NaT,NaT,,,PJ/Autônomo,25364,,(21) 96332-6963,Liderança / Desenvolvimento,17-03-2021 09:41:10,Juliana Cassiano,sra._yasmin_fernandes@hotmail.com,"São Paulo, São Paulo",Site de Empregos,12-04-2021 12:15:57,25364,Sra. Yasmin Fernandes,Cadastro anterior ao registro de aceite,Sra. Yasmin Fernandes,,Site de Empregos: APINFO / Linkedin,sra._yasmin_fernandes@hotmail.com,,31-01-1973,(21) 96332-6963,Masculino,Casado,Não,são paulo,,,,Liderança / Desenvolvimento,TI - Projetos,"- PeopleSoft (PeopleTools 8.49

In [28]:
features_palavras_chaves = {
                 'key_SAP': r'SAP',
                 'key_SD': r'\bSD\b',
                 'key_MM': r'\bMM\b',
                 'key_ABAP': r'\bABAP\b',
                 'key_AMS': r'AMS',
                 'key_JAVA': r'java',
                 'key_ORACLE': r'oracle',
                 'key_CLOUD': r'cloud|AWS|Azure',
                 'key_EBS': r'\bEBS\b',
                 'key_DBA': r'\bDBA\b',
                 'key_PROXXI': r'\bPROXXI\b',
                 'key_C_HASH': r'C#',
                 'key_OPERATIONS': r'Operations|Operações',
                 'key_PMO': r'\bPMO\b|projetos',
                 'key_ADM': r'\b(Adm|Administ|Administrativo|Administrador)\b',
                 'key_MARKETING': r'market',
                 'key_TI': r'\bTI\b',
                 'key_SALESFORCE': r'Salesforce',
                 'key_PROJETOS': r'Projetos',
                 'key_DADOS': r'Dados|Data',
                 'key_SECURITY': r'\b(Segurança|Security|Cyber)\b',
                 'key_WEB': r'\bWeb\b',
                 'key_SCRUM': r'\bScrum\b',
                 'key_SERVICE': r'Service',
                 'key_REACT_NATIVE_ANGULAR': r'\b(react|native|angular)',
                 'key_NET': r'.Net',
                 'key_Analista': r'Analista|Analyst',
                 'key_DEVOPS': r'devops',
                 'key_PYTHON': r'\bpython\b',
                 'key_C_PLUS_PLUS': r'C\+\+',
                 #'key_DEV': r'(Desenvolvedor|Developer|Dev)',
                 'key_COBOL': r'Cobol',
                 'key_ANDROID': r'Android',
                 'key_SQL': r'SQL',
                 'key_fiscal': r'fiscal|contábil|contabilidade',
                 'key_software': r'Software|Front|FullStack|back'
}

In [29]:
# Criação de colunas binárias que informam palavras chave ou não
for coluna, regex_pattern in features_palavras_chaves.items():
    final_df[coluna] = final_df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)

C:\Users\renan_dyoksic\AppData\Local\Temp\ipykernel_14560\4055058583.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  final_df[coluna] = final_df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)
C:\Users\renan_dyoksic\AppData\Local\Temp\ipykernel_14560\4055058583.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  final_df[coluna] = final_df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)
C:\Users\renan_dyoksic\AppData\Local\Temp\ipykernel_14560\4055058583.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  final_df[coluna] = final_df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)


In [79]:
# Total de prospecções que sobraram após filtro de Matches
final_df.shape

(8173, 114)

In [30]:
# Total de prospecções que não teve nenhum valor "1" nas colunas binárias criadas
final_df[final_df[list(features_palavras_chaves.keys())].eq(0).all(axis=1)].shape

(1611, 149)

In [23]:
# Substitui df por apenas as linhas com "0" em todas as colunas binárias
final_df = final_df[final_df[list(features_palavras_chaves.keys())].eq(0).all(axis=1)]

In [ ]:
# SAP tem sub opções de consultor, SD, MM, ABAP, AMS
# Java
# Oracle possui subs de : Cloud, EBS, DBA, SQL
# PROXXI subs: C#
# Operations subs: PMO
# Adm. subs: Administrativo, Administrador
# Marketing
# TI subs: 
# Salesforce
# Projetos
# Dados: subs: Cientista
# Segurança subs: Cyber, Security
# Web
# Scrum
# Service
# React, native, ou angular
# Analista
# DevOps
# Python
# C++
# Desenvolvedor ou Developer ou Dev
# Cobol
# Android
# SQL

In [24]:
# Conta palavras mais repetidas em toda a coluna de título da prospecção
texto = " ".join(final_df['prospects_titulo'].dropna()).split()
Counter(texto)

Counter({'-': 759,
         'de': 235,
         'Consultor': 213,
         'Desenvolvedor': 195,
         'Developer': 97,
         'Senior': 94,
         'Dev': 78,
         'Pleno': 65,
         '/': 63,
         'End': 62,
         'Sênior': 57,
         'Engineer': 55,
         'QA': 54,
         'Suporte': 53,
         'Sr': 53,
         'Tech': 49,
         'Product': 43,
         'Owner': 43,
         'Full': 41,
         'SR': 41,
         'Front': 40,
         'Manager': 40,
         'Técnico': 40,
         'Quality': 38,
         'Arquiteto': 37,
         'Lead': 36,
         'Stack': 36,
         'Gerente': 35,
         'Software': 34,
         'Especialista': 33,
         'e': 33,
         'Assistente': 33,
         '–': 33,
         'Junior': 31,
         'Test': 30,
         'PL': 30,
         'Frontend': 30,
         'Telecom': 30,
         'Automation': 29,
         'Backend': 29,
         'Estoque': 28,
         'Power': 27,
         'Assurance': 26,
         'JDE': 24

In [13]:
# Contagem de palavras que estejam em linhas que possuem a palavra declarada para a função
palavras_associadas(final_df, 'prospects_titulo', r'scrum')

([], Series([], dtype: object))

# Deletar Seção de teste de classes e funções

In [145]:
import os

base_path = os.getcwd()

In [146]:
import pandas as pd
import json

class DataLoader:
    """Classe para carregar dados de candidatos, vagas e prospecção a partir de arquivos JSON."""

    def __init__(self, base_path: str):
        """
        Inicializa a classe com o caminho base onde estão os arquivos JSON.

        :param base_path: Caminho do diretório contendo os arquivos JSON.
        """
        self.base_path = base_path

    def load_applicants(self, file_name: str) -> pd.DataFrame:
        """
        Carrega candidatos a partir de um arquivo JSON e converte para um DataFrame do pandas.

        :param file_name: Nome do arquivo JSON dentro do diretório base.
        :return: DataFrame com os dados dos candidatos.
        """
        file_path = f"{self.base_path}/{file_name}"

        try:
            with open(file_path, "r", encoding="utf-8") as file:
                data = json.load(file)

            applicants_list = []

            for appl_id, applicant in data.items():
                linha = {"id_applicants": appl_id}

                # Variáveis dentro da chave infos_basicas
                infos_basicas = applicant.get("infos_basicas", {})
                linha.update({
                    "telefone_recado": infos_basicas.get("telefone_recado", ""),
                    "telefone": infos_basicas.get("telefone", ""),
                    "infos_basicas_objetivo_profissional": infos_basicas.get("objetivo_profissional", ""),
                    "infos_basicas_data_criacao": infos_basicas.get("data_criacao", ""),
                    "infos_basicas_inserido_por": infos_basicas.get("inserido_por", ""),
                    "infos_basicas_email": infos_basicas.get("email", ""),
                    "infos_basicas_local": infos_basicas.get("local", ""),
                    "infos_basicas_sabendo_de_nos_por": infos_basicas.get("sabendo_de_nos_por", ""),
                    "infos_basicas_data_atualizacao": infos_basicas.get("data_atualizacao", ""),
                    "infos_basicas_codigo_profissional": infos_basicas.get("codigo_profissional", ""),
                    "infos_basicas_nome": infos_basicas.get("nome", "")
                })

                # Variáveis dentro da chave informacoes_pessoais
                pessoais = applicant.get("informacoes_pessoais", {})
                linha.update({
                    "data_aceite": pessoais.get("data_aceite", ""),
                    "nome": pessoais.get("nome", ""),
                    "cpf": pessoais.get("cpf", ""),
                    "fonte_indicacao": pessoais.get("fonte_indicacao", ""),
                    "email": pessoais.get("email", ""),
                    "email_secundario": pessoais.get("email_secundario", ""),
                    "data_nascimento": pessoais.get("data_nascimento", ""),
                    "telefone_celular": pessoais.get("telefone_celular", ""),
                    "telefone_recado": pessoais.get("telefone_recado", ""),
                    "sexo": pessoais.get("sexo", ""),
                    "estado_civil": pessoais.get("estado_civil", ""),
                    "pcd": pessoais.get("pcd", ""),
                    "endereco": pessoais.get("endereco", ""),
                    "skype": pessoais.get("skype", ""),
                    "url_linkedin": pessoais.get("url_linkedin", ""),
                    "facebook": pessoais.get("facebook", "")
                })

                # Variáveis dentro da chave informacoes_profissionais
                profissionais = applicant.get("informacoes_profissionais", {})
                linha.update({
                    "titulo_profissional": profissionais.get("titulo_profissional", ""),
                    "area_atuacao": profissionais.get("area_atuacao", ""),
                    "conhecimentos_tecnicos": profissionais.get("conhecimentos_tecnicos", ""),
                    "certificacoes": profissionais.get("certificacoes", ""),
                    "outras_certificacoes": profissionais.get("outras_certificacoes", ""),
                    "remuneracao": profissionais.get("remuneracao", ""),
                    "nivel_profissional": profissionais.get("nivel_profissional", "")
                })

                # Variáveis dentro da chave formacao_e_idiomas
                formacao = applicant.get("formacao_e_idiomas", {})
                linha.update({
                    "nivel_academico": formacao.get("nivel_academico", ""),
                    "nivel_ingles": formacao.get("nivel_ingles", ""),
                    "nivel_espanhol": formacao.get("nivel_espanhol", ""),
                    "outro_idioma": formacao.get("outro_idioma", ""),
                    "instituicao_ensino_superior": formacao.get("instituicao_ensino_superior", ""),
                    "cursos": formacao.get("cursos", ""),
                    "ano_conclusao": formacao.get("ano_conclusao", ""),
                    "download_cv": formacao.get("download_cv", ""),
                    "qualificacoes": formacao.get("qualificacoes", ""),
                    "experiencias": formacao.get("experiencias", ""),
                    "outro_curso": formacao.get("outro_curso", "")
                })

                # Variáveis dentro da chave cargo_atual
                cargo = applicant.get("cargo_atual", {})
                linha.update({
                    "id_ibrati": cargo.get("id_ibrati", ""),
                    "email_corporativo": cargo.get("email_corporativo", ""),
                    "cargo_atual": cargo.get("cargo_atual", ""),
                    "projeto_atual": cargo.get("projeto_atual", ""),
                    "cliente": cargo.get("cliente", ""),
                    "unidade": cargo.get("unidade", ""),
                    "data_admissao": cargo.get("data_admissao", ""),
                    "data_ultima_promocao": cargo.get("data_ultima_promocao", ""),
                    "nome_superior_imediato": cargo.get("nome_superior_imediato", ""),
                    "email_superior_imediato": cargo.get("email_superior_imediato", "")
                })

                # Variáveis dentro da chave cv
                linha["cv_pt"] = applicant.get("cv_pt", "")
                linha["cv_en"] = applicant.get("cv_en", "")

                applicants_list.append(linha)

            return pd.DataFrame(applicants_list)

        except (FileNotFoundError, json.JSONDecodeError) as e:
            print(f"Erro ao carregar candidatos: {e}")
            return pd.DataFrame()

    def load_vagas(self, file_name: str) -> pd.DataFrame:
        """
        Carrega vagas a partir de um arquivo JSON e converte para um DataFrame do pandas.

        :param file_name: Nome do arquivo JSON dentro do diretório base.
        :return: DataFrame com os dados das vagas.
        """
        file_path = f"{self.base_path}/{file_name}"

        try:
            with open(file_path, "r", encoding="utf-8") as file:
                data = json.load(file)

            vagas_list = []

            for vaga_id, vaga in data.items():
                linha = {"id_vaga": vaga_id}

                # Variáveis dentro da chave informacoes_basicas
                info_basicas = vaga.get("informacoes_basicas", {})
                linha.update({
                    "data_requicisao": info_basicas.get("data_requicisao", ""),
                    "limite_esperado_para_contratacao": info_basicas.get("limite_esperado_para_contratacao", ""),
                    "titulo_vaga": info_basicas.get("titulo_vaga", ""),
                    "vaga_sap": info_basicas.get("vaga_sap", ""),
                    "cliente": info_basicas.get("cliente", ""),
                    "solicitante_cliente": info_basicas.get("solicitante_cliente", ""),
                    "empresa_divisao": info_basicas.get("empresa_divisao", ""),
                    "requisitante": info_basicas.get("requisitante", ""),
                    "analista_responsavel": info_basicas.get("analista_responsavel", ""),
                    "tipo_contratacao": info_basicas.get("tipo_contratacao", ""),
                    "prazo_contratacao": info_basicas.get("prazo_contratacao", ""),
                    "objetivo_vaga": info_basicas.get("objetivo_vaga", ""),
                    "prioridade_vaga": info_basicas.get("prioridade_vaga", ""),
                    "origem_vaga": info_basicas.get("origem_vaga", ""),
                    "superior_imediato": info_basicas.get("superior_imediato", "")
                })

                # Variáveis dentro da chave perfil_vaga
                perfil = vaga.get("perfil_vaga", {})
                linha.update({
                    "nome": perfil.get("nome", ""),
                    "telefone": perfil.get("telefone", ""),
                    "pais": perfil.get("pais", ""),
                    "estado": perfil.get("estado", ""),
                    "cidade": perfil.get("cidade", ""),
                    "bairro": perfil.get("bairro", ""),
                    "regiao": perfil.get("regiao", ""),
                    "local_trabalho": perfil.get("local_trabalho", ""),
                    "vaga_especifica_para_pcd": perfil.get("vaga_especifica_para_pcd", ""),
                    "faixa_etaria": perfil.get("faixa_etaria", ""),
                    "horario_trabalho": perfil.get("horario_trabalho", ""),
                    "nivel profissional": perfil.get("nivel profissional", ""),
                    "nivel_academico": perfil.get("nivel_academico", ""),
                    "nivel_ingles": perfil.get("nivel_ingles", ""),
                    "nivel_espanhol": perfil.get("nivel_espanhol", ""),
                    "outro_idioma": perfil.get("outro_idioma", ""),
                    "areas_atuacao": perfil.get("areas_atuacao", ""),
                    "principais_atividades": perfil.get("principais_atividades", ""),
                    "competencia_tecnicas_e_comportamentais": perfil.get("competencia_tecnicas_e_comportamentais", ""),
                    "demais_observacoes": perfil.get("demais_observacoes", ""),
                    "viagens_requeridas": perfil.get("viagens_requeridas", ""),
                    "equipamentos_necessarios": perfil.get("equipamentos_necessarios", "")
                })

                # Variáveis dentro da chave beneficios
                beneficios = vaga.get("beneficios", {})
                linha.update({
                    "valor_venda": beneficios.get("valor_venda", ""),
                    "valor_compra_1": beneficios.get("valor_compra_1", ""),
                    "valor_compra_2": beneficios.get("valor_compra_2", ""),
                    "data_inicial": beneficios.get("data_inicial", ""),
                    "data_final": beneficios.get("data_final", ""),
                    "habilidades_comportamentais_necessarias": beneficios.get("habilidades_comportamentais_necessarias", ""),
                    "nome_substituto": beneficios.get("nome_substituto", "")
                })
    
                vagas_list.append(linha)

            return pd.DataFrame(vagas_list)

        except (FileNotFoundError, json.JSONDecodeError) as e:
            print(f"Erro ao carregar vagas: {e}")
            return pd.DataFrame()
        
    def load_prospects(self, file_name: str) -> pd.DataFrame:
        """
        Carrega prospecções a partir de um arquivo JSON e converte para um DataFrame do pandas.

        :param file_name: Nome do arquivo JSON dentro do diretório base.
        :return: DataFrame com os dados das prospecções.
        """
        file_path = f"{self.base_path}/{file_name}"

        try:
            with open(file_path, "r", encoding="utf-8") as file:
                data = json.load(file)

            prospects_list = []

            for vaga_id, vaga in data.items():
                titulo = vaga.get("titulo", "")
                modalidade = vaga.get("modalidade", "")
                prospects = vaga.get("prospects", [])

                for prospect in prospects:
                    linha = {
                        "vaga_id": vaga_id,
                        "titulo": titulo,
                        "modalidade": modalidade,
                        "quantidade_prospects": len(prospects),
                        "nome": prospect.get("nome", ""),
                        "codigo": prospect.get("codigo", ""),
                        "situacao_candidado": prospect.get("situacao_candidado", ""),
                        "data_candidatura": prospect.get("data_candidatura", ""),
                        "ultima_atualizacao": prospect.get("ultima_atualizacao", ""),
                        "comentario": prospect.get("comentario", ""),
                        "recrutador": prospect.get("recrutador", "")
                    }
                    prospects_list.append(linha)

            return pd.DataFrame(prospects_list)

        except (FileNotFoundError, json.JSONDecodeError) as e:
            print(f"Erro ao carregar prospects: {e}")
            return pd.DataFrame()

In [147]:
import pandas as pd
from typing import List


class CleanApplicants:
    """Classe para realizar a limpeza e conversão de tipos no DataFrame de aplicantes."""

    def __init__(self, df: pd.DataFrame):
        """
        Inicializa a classe com um DataFrame.

        :param df: DataFrame contendo os dados dos aplicantes.
        """
        self.df = df.copy()
        
        # Colunas que devem ser tratadas como tipo texto
        self.colunas_str = [
            "telefone_recado", "telefone", "infos_basicas_objetivo_profissional",
            "infos_basicas_inserido_por", "infos_basicas_email", "infos_basicas_local",
            "infos_basicas_sabendo_de_nos_por", "infos_basicas_nome", "data_aceite",
            "nome", "cpf", "fonte_indicacao",
            "email", "email_secundario", "telefone_celular",
            "sexo", "estado_civil", "pcd",
            "endereco", "skype", "url_linkedin",
            "facebook", "titulo_profissional", "area_atuacao",
            "conhecimentos_tecnicos", "certificacoes", "outras_certificacoes",
            "nivel_profissional", "nivel_academico", "nivel_ingles",
            "nivel_espanhol", "outro_idioma", "instituicao_ensino_superior",
            "cursos", "download_cv", "qualificacoes",
            "experiencias", "outro_curso", "email_corporativo",
            "cargo_atual", "projeto_atual", "cliente",
            "unidade", "nome_superior_imediato", "email_superior_imediato",
            "cv_pt", "cv_en"
        ]
        
        # Colunas que devem ser tratadas como tipo data
        self.colunas_data = [
            "infos_basicas_data_criacao", "infos_basicas_data_atualizacao", "data_nascimento",
            "data_admissao", "data_ultima_promocao"
        ]


        # Colunas que devem ser tratadas como tipo inteiro
        self.colunas_int = [
            "infos_basicas_codigo_profissional", "id_applicants", "id_ibrati"
        ]
        
    def clean_remuneracao(self) -> None:
        """Limpa e converte a coluna 'remuneracao' para float."""
        self.df["remuneracao"] = (
            self.df["remuneracao"]
            .astype(str)
            .str.lower()
            .str.extract(r"mensal\s*/\s*([\d.,]+)", expand=False)  # Captura o número após 'mensal /'
            .fillna("0")  # Se não encontrar, substitui por 0
            .str.replace(".", "", regex=False)  # Remove pontos
            .str.replace(",", ".", regex=False)  # Substitui vírgula por ponto
            .astype(float)
        )

    def convert_to_string(self) -> None:
        """Converte as colunas especificadas para string."""
        for col in self.colunas_str:
            if col in self.df.columns:
                self.df[col] = self.df[col].astype(str)

    def convert_to_datetime(self) -> None:
        """Converte as colunas especificadas para datetime."""
        for col in self.colunas_data:
            if col in self.df.columns:
                self.df[col] = pd.to_datetime(self.df[col], errors="coerce", dayfirst=True)

    def convert_to_int(self) -> None:
        """Converte as colunas especificadas para int."""
        for col in self.colunas_int:
            if col in self.df.columns:
                self.df[col] = pd.to_numeric(self.df[col], errors="coerce").fillna(0).astype(int)

    def clean_all(self) -> pd.DataFrame:
        """Executa todas as funções de limpeza."""
        self.clean_remuneracao()
        self.convert_to_string()
        #self.convert_to_datetime()
        self.convert_to_int()

        return self.df

class CleanVagas:
    """Classe para realizar a limpeza e conversão de tipos no DataFrame de vagas."""

    def __init__(self, df: pd.DataFrame):
        """
        Inicializa a classe com um DataFrame.

        :param df: DataFrame contendo os dados das vagas.
        """
        self.df = df.copy()
        
        # Colunas que devem ser tratadas como tipo texto
        self.colunas_string = [
            'titulo_vaga', 'vaga_sap', 'cliente', 'solicitante_cliente',
            'empresa_divisao', 'requisitante', 'analista_responsavel',
            'tipo_contratacao', 'prazo_contratacao', 'objetivo_vaga',
            'prioridade_vaga', 'origem_vaga', 'superior_imediato',
            'nome', 'telefone', 'pais', 'estado', 'cidade', 'bairro',
            'regiao', 'local_trabalho', 'vaga_especifica_para_pcd',
            'faixa_etaria', 'horario_trabalho', 'nivel profissional', 'nivel_academico',
            'nivel_ingles', 'nivel_espanhol', 'outro_idioma', 'areas_atuacao',
            'principais_atividades', 'competencia_tecnicas_e_comportamentais',
            'demais_observacoes', 'viagens_requeridas', 'equipamentos_necessarios',
            'valor_compra_1', 'valor_compra_2', 'habilidades_comportamentais_necessarias',
            'nome_substituto'
        ]
        
        
        # Colunas que devem ser tratadas como tipo data
        self.colunas_data = [
            'data_requicisao', 'limite_esperado_para_contratacao',
            'data_inicial', 'data_final'
        ]

        # Colunas que devem ser tratadas como tipo data
        self.colunas_int = [
            'id_vaga'
        ]
        
    @staticmethod
    def limpar_valor(valor: str) -> float:
        """Limpa e converte valores monetários para float."""
        valor = str(valor).strip()
        if valor == '-' or 'p/ mês' in valor or '(' in valor or valor.count('.') > 1:
            return 0.0

        # Remove caracteres indesejados e converte para formato numérico válido
        valor = (
            valor.replace("R$", "")
            .replace(".", "")  # Remove pontos de milhar
            .replace(",", ".")  # Substitui vírgula decimal por ponto
        )
        valor = "".join(c for c in valor if c.isdigit() or c == ".")

        try:
            return float(valor)
        except:
            return 0.0

    def clean_valor_venda(self) -> None:
        """Aplica a limpeza à coluna 'valor_venda'."""
        self.df["valor_venda"] = self.df["valor_venda"].apply(self.limpar_valor)

    def convert_to_datetime(self) -> None:
        """Converte colunas especificadas para formato datetime."""
        for col in self.colunas_data:
            if col in self.df.columns:
                self.df[col] = pd.to_datetime(self.df[col], format="%d-%m-%Y", errors="coerce")

    def convert_to_string(self) -> None:
        """Converte colunas especificadas para string."""
        for col in self.colunas_string:
            if col in self.df.columns:
                self.df[col] = self.df[col].astype(str)

    def convert_to_int(self) -> None:
        """Converte a coluna especificada para inteiro."""
        for col in self.colunas_int:
            if col in self.df.columns:
                self.df[col] = pd.to_numeric(self.df[col], errors="coerce").astype("Int64")

    @staticmethod
    def categorizar_contratacao(tipo: str) -> str:
        """Cria categorias agrupadas para tipos de contratação."""
        categorias = set()
        if not tipo or tipo.strip() == "":
            return "Vazio/Indefinido"
        
        if "CLT Full" in tipo or "CLT Cotas" in tipo:
            categorias.add("CLT")
        if "PJ/Autônomo" in tipo:
            categorias.add("PJ/Autônomo")
        if "Cooperado" in tipo:
            categorias.add("Cooperado")
        if "Estagiário" in tipo:
            categorias.add("Estágio")
        if "Hunting" in tipo:
            categorias.add("Hunting")
        if "Candidato poderá escolher" in tipo:
            categorias.add("Escolha do Candidato")
        
        return ", ".join(sorted(categorias))

    def clean_tipo_contratacao(self) -> None:
        """Aplica a categorização de tipos de contratação."""
        self.df["categoria_contratacao"] = self.df["tipo_contratacao"].apply(self.categorizar_contratacao)

    def clean_all(self) -> pd.DataFrame:
        """Executa todas as funções de limpeza."""
        self.clean_valor_venda()
        self.clean_tipo_contratacao()
        self.convert_to_datetime()
        self.convert_to_string()
        self.convert_to_int()

        return self.df


class CleanProspects:
    """Classe para realizar a limpeza e conversão de tipos no DataFrame de prospects."""

    def __init__(self, df: pd.DataFrame):
        """
        Inicializa a classe com um DataFrame.

        :param df: DataFrame contendo os dados das prospecções.
        """
        self.df = df.copy()
        
        # Colunas que devem ser tratadas como tipo texto
        self.colunas_string = [
            "titulo", "modalidade", "nome", "situacao_candidado", "comentario", "recrutador"
        ]
        
        # Colunas que devem ser tratadas como tipo data
        self.colunas_data = [
            'data_candidatura', 'ultima_atualizacao'
        ]

        # Colunas que devem ser tratadas como tipo data
        self.colunas_int = [
            'vaga_id', 'quantidade_prospects', 'codigo'
        ]

    def convert_to_datetime(self) -> None:
        """Converte colunas especificadas para formato datetime."""
        for col in self.colunas_data:
            if col in self.df.columns:
                self.df[col] = pd.to_datetime(self.df[col], format="%d-%m-%Y", errors="coerce")

    def convert_to_string(self) -> None:
        """Converte colunas especificadas para string."""
        for col in self.colunas_string:
            if col in self.df.columns:
                self.df[col] = self.df[col].astype(str)

    def convert_to_int(self) -> None:
        """Converte a coluna especificada para inteiro."""
        for col in self.colunas_int:
            if col in self.df.columns:
                self.df[col] = pd.to_numeric(self.df[col], errors="coerce").astype("Int64")

    def clean_all(self) -> pd.DataFrame:
        """Executa todas as funções de limpeza."""
        self.convert_to_datetime()
        self.convert_to_string()
        self.convert_to_int()
        
        return self.df

In [155]:
def data_unify(applicants, prospects, vagas):
    
    
    applicants = applicants.add_prefix("applicants_")
    prospects = prospects.add_prefix("prospects_")
    vagas = vagas.add_prefix("vagas_")
    
    united_data = pd.merge(
        prospects,
        vagas,
        left_on="prospects_vaga_id",
        right_on="vagas_id_vaga",
        how="inner"
    )
    
    united_data = pd.merge(
        united_data,
        applicants,
        left_on="prospects_codigo",
        right_on="applicants_infos_basicas_codigo_profissional",
        how="inner"
    )
    
    united_data = united_data[united_data['prospects_situacao_candidado'].isin([
        "Não Aprovado pelo Cliente",
        "Contratado pela Decision",
        "Não Aprovado pelo RH",
        "Não Aprovado pelo Requisitante",
        "Sem interesse nesta vaga",
        "Contratado como Hunting",
        "Aprovado",
        "Recusado",
        "Proposta Aceita"
        ])
    ]
    
    mapa_target = {
                'Não Aprovado pelo Cliente': 0,
                'Contratado pela Decision': 1,
                'Não Aprovado pelo RH': 0,
                'Não Aprovado pelo Requisitante': 0,
                'Sem interesse nesta vaga': 0,
                'Contratado como Hunting': 1,
                'Aprovado': 1,
                'Recusado': 0,
                'Proposta Aceita': 1
            }
        
    # Aplicar o mapeamento em novas colunas
    united_data['prospects_situacao_candidado'] = united_data['prospects_situacao_candidado'].map(mapa_target)
    united_data = united_data.rename(columns= {'prospects_situacao_candidado': 'target'})
    united_data = united_data.set_index('applicants_id_applicants')

    return united_data

In [149]:
class vagas_features_engineering:
    
    def __init__(self, df: pd.DataFrame):
        """
        Inicializa a classe com um DataFrame.

        :param df: DataFrame contendo os dados das vagas.
        """
        self.df = df.copy()
        self.input_column_names = df.columns
        
    def titulo_vaga(self) -> None:
        """Descrição aqui"""
        
        mapa_palavras_chave_titulo = {
            'key_SAP': r'SAP',
            'key_SD': r'\bSD\b',
            'key_MM': r'\bMM\b',
            'key_ABAP': r'\bABAP\b',
            'key_AMS': r'AMS',
            'key_JAVA': r'java',
            'key_ORACLE': r'oracle',
            'key_CLOUD': r'cloud|AWS|Azure',
            'key_EBS': r'\bEBS\b',
            'key_DBA': r'\bDBA\b',
            'key_PROXXI': r'\bPROXXI\b',
            'key_C_HASH': r'C#',
            'key_OPERATIONS': r'Operations|Operações',
            'key_PMO': r'\bPMO\b|projetos',
            'key_ADM': r'\b(Adm|Administ|Administrativo|Administrador)\b',
            'key_MARKETING': r'market',
            'key_TI': r'\bTI\b',
            'key_SALESFORCE': r'Salesforce',
            'key_PROJETOS': r'Projetos',
            'key_DADOS': r'Dados|Data',
            'key_SECURITY': r'\b(Segurança|Security|Cyber)\b',
            'key_WEB': r'\bWeb\b',
            'key_SCRUM': r'\bScrum\b',
            'key_SERVICE': r'Service',
            'key_REACT_NATIVE_ANGULAR': r'\b(react|native|angular)',
            'key_NET': r'.Net',
            'key_Analista': r'Analista|Analyst',
            'key_DEVOPS': r'devops',
            'key_PYTHON': r'\bpython\b',
            'key_C_PLUS_PLUS': r'C\+\+',
            'key_COBOL': r'Cobol',
            'key_ANDROID': r'Android',
            'key_SQL': r'SQL',
            'key_fiscal': r'fiscal|contábil|contabilidade',
            'key_software': r'Software|Front|FullStack|back'
        }
        
        
        # Criação de colunas binárias que informam palavras chave ou não
        for coluna, regex_pattern in mapa_palavras_chave_titulo.items():
            self.df[coluna] = self.df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)

    @staticmethod
    def map_nivel(titulo: str)-> str:
        niveis_dict = {
            "Estágio/Trainee": r"estagi|trainee",
            "Assistente": r"assistente",
            "Júnior": r"junior|\bjr\b",
            "Pleno": r"pleno|\bpl\b",
            "Sênior": r"senior|\bsr\b|sênior|\bsn\b",
            "Coordenação": r"coordenador|líder",
            "Gerência": r"gerente|manager",
            "Diretoria": r"diretor"
        }
        
        for nivel, padrao in niveis_dict.items():
            if re.search(padrao, titulo):
                return nivel
            
        return "Outros"
    
    def nivel_vaga(self) -> None:
        """Descrição aqui"""
        
        mapeamento_numerico = {
            "Outros": 0,
            "Estágio/Trainee": 0,
            "Assistente": 1,
            "Júnior": 2,
            "Pleno": 3,
            "Sênior": 4,
            "Coordenação": 5,
            "Gerência": 6,
            "Diretoria": 7
        }
        
        self.df['nivel_vaga'] = self.df['prospects_titulo'].apply(self.map_nivel)
        self.df['nivel_vaga'] = self.df['nivel_vaga'].map(mapeamento_numerico)
        
    def tipo_contratacao(self) -> None:
        """Descrição aqui"""
        
        mapa_palavras_chave_tipo = {
            'tipo_escolha_do_candidato': r'candidato poderá escolher',
            'tipo_CLT': r'clt',
            'tipo_PJ': r'pj',
            'tipo_cooperado': r'cooperado',
            'tipo_estagio': r'estagiário',
            'tipo_hunting': r'hunting',
            'tipo_misto': r'^.*[,]|(?=.*\bclt\b)(?=.*\bpj\b)(?=.*\bcooperado\b)(?=.*\bestagiário\b)(?=.*\bhunting\b).*',
        }
        
        # Criação de colunas binárias que informam palavras chave ou não
        for coluna, regex_pattern in mapa_palavras_chave_tipo.items():
            self.df[coluna] = self.df['vagas_tipo_contratacao'].str.contains(regex_pattern, regex= True, case= False).astype(int)
        
        
    def linguas_vaga(self) -> None:
        """Descrição aqui"""
        
        mapa_linguas = {
            'Nenhum': 0,
            'Básico': 1,
            'Intermediário': 2,
            'Avançado': 3,
            'Técnico': 3,
            'Fluente': 4
        }
        
        # Aplicar o mapeamento em novas colunas
        self.df['score_ingles'] = self.df['vagas_nivel_ingles'].map(mapa_linguas)
        self.df['score_espanhol'] = self.df['vagas_nivel_espanhol'].map(mapa_linguas).fillna(int(0))
        
    def create_all(self) -> pd.DataFrame:
        
        self.titulo_vaga()
        self.nivel_vaga()
        self.tipo_contratacao()
        self.linguas_vaga()
        
        return self.df.drop(columns=self.input_column_names, errors='ignore')

In [165]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler

def normalize_dataframe(df, method='minmax'):

    # Escolhe o scaler com base no método
    if method == 'minmax':
        scaler = MinMaxScaler()
    elif method == 'standard':
        scaler = StandardScaler()
    elif method == 'robust':
        scaler = RobustScaler()
    elif method == 'maxabs':
        scaler = MaxAbsScaler()
    else:
        raise ValueError("Método inválido. Escolha entre: 'minmax', 'standard', 'robust', 'maxabs'.")

    return pd.DataFrame(scaler.fit_transform(df), columns= df.columns, index= df.index)

In [151]:
loader = DataLoader(base_path)
prospects = loader.load_prospects("prospects.json")
vagas = loader.load_vagas("vagas.json")
aplicants = loader.load_applicants("applicants.json")

In [156]:
# Exemplo de uso:
prospects = CleanProspects(prospects).clean_all()
vagas = CleanVagas(vagas).clean_all()
aplicants = CleanApplicants(aplicants).clean_all()

In [168]:
df_unido = data_unify(aplicants, prospects, vagas)

In [174]:
df_unido_gerado = vagas_features_engineering(df_unido).create_all()

C:\Users\renan_dyoksic\AppData\Local\Temp\ipykernel_9804\3326073744.py:56: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  self.df[coluna] = self.df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)
C:\Users\renan_dyoksic\AppData\Local\Temp\ipykernel_9804\3326073744.py:56: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  self.df[coluna] = self.df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)
C:\Users\renan_dyoksic\AppData\Local\Temp\ipykernel_9804\3326073744.py:56: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  self.df[coluna] = self.df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)


In [170]:
normalize_dataframe(df_unido_gerado, method='minmax')

,key_SAP,key_SD,key_MM,key_ABAP,key_AMS,key_JAVA,key_ORACLE,key_CLOUD,key_EBS,key_DBA,key_PROXXI,key_C_HASH,key_OPERATIONS,key_PMO,key_ADM,key_MARKETING,key_TI,key_SALESFORCE,key_PROJETOS,key_DADOS,key_SECURITY,key_WEB,key_SCRUM,key_SERVICE,key_REACT_NATIVE_ANGULAR,key_NET,key_Analista,key_DEVOPS,key_PYTHON,key_C_PLUS_PLUS,key_COBOL,key_ANDROID,key_SQL,key_fiscal,key_software,nivel_vaga,tipo_escolha_do_candidato,tipo_CLT,tipo_PJ,tipo_cooperado,tipo_estagio,tipo_hunting,tipo_misto,score_ingles,score_espanhol
applicants_id_applicants,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
25364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.00,0.0
26338,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.75,1.0
26003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.75,0.0
12819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.75,0.0
8838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.75,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35823,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.75,0.0
7952,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.25,0.0
14167,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.75,0.0


In [171]:
import pandas as pd
from sentence_transformers import SentenceTransformer

def applicants_features_engineering(df: pd.DataFrame, description_column: str) -> pd.DataFrame:

    text_series = df[description_column]
    # Using a smaller model suitable for Portuguese
    model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
    embeddings = model.encode(text_series.tolist())

    # Create a DataFrame from the embeddings
    embedding_df = pd.DataFrame(embeddings, columns= [f'embedd_{i}' for i in range(1, 512 + 1)], index= df.index)

    return embedding_df

    # Example Usage:
data = {'phrases': ['This is the first sentence.', 'This is the second sentence.', 'And this is the third sentence.']}
df = pd.DataFrame(data)

embeddings_df = applicants_features_engineering(df, 'phrases')
print(embeddings_df.head())

   embedd_1  embedd_2  embedd_3  embedd_4  embedd_5  embedd_6  embedd_7  \
0  0.041003  0.042992 -0.003132  0.083645 -0.086153 -0.019330  0.028382   
1  0.006209  0.025756  0.011027  0.071507 -0.081832 -0.003358  0.034962   
2  0.018494 -0.012826 -0.040713  0.070955 -0.095126  0.000493  0.060057   

   embedd_8  embedd_9  embedd_10  embedd_11  embedd_12  embedd_13  embedd_14  \
0  0.027023  0.023653  -0.011299   0.025794  -0.030526  -0.009810   0.004396   
1 -0.005530 -0.015574  -0.014661  -0.015207  -0.026648   0.013252   0.014289   
2  0.048090  0.018739  -0.040358   0.001058  -0.024787   0.032499  -0.000156   

   embedd_15  embedd_16  embedd_17  embedd_18  embedd_19  embedd_20  \
0  -0.052944  -0.031909  -0.020006  -0.006944  -0.034838   0.036978   
1  -0.073345  -0.013605  -0.008696  -0.051390  -0.063823  -0.000019   
2  -0.021375   0.011420  -0.004815  -0.042201  -0.026223  -0.007523   

   embedd_21  embedd_22  embedd_23  embedd_24  embedd_25  embedd_26  \
0  -0.020833   0.08191

In [182]:
pd.concat([df_unido_gerado.tail(60), applicants_features_engineering(df_unido.tail(60), 'applicants_cv_pt'), df_unido['target'].tail(60)], axis= 1).drop(columns= 'target').to_parquet('df_to_model_predict.parquet')

In [193]:
loader = DataLoader(base_path)
prospects = loader.load_prospects("prospects.json")
vagas = loader.load_vagas("vagas.json")
aplicants = loader.load_applicants("applicants.json")

prospects = CleanProspects(prospects).clean_all()
vagas = CleanVagas(vagas).clean_all()
aplicants = CleanApplicants(aplicants).clean_all()

df_unido = data_unify(aplicants, prospects, vagas)

df_unido_gerado = vagas_features_engineering(df_unido).create_all()
    
pd.concat([df_unido_gerado.head(),
               applicants_features_engineering(df_unido.head(), 'applicants_cv_pt'),
               df_unido['target'].head()], axis= 1
)

C:\Users\renan_dyoksic\AppData\Local\Temp\ipykernel_9804\3326073744.py:56: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  self.df[coluna] = self.df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)
C:\Users\renan_dyoksic\AppData\Local\Temp\ipykernel_9804\3326073744.py:56: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  self.df[coluna] = self.df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)
C:\Users\renan_dyoksic\AppData\Local\Temp\ipykernel_9804\3326073744.py:56: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  self.df[coluna] = self.df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)


,key_SAP,key_SD,key_MM,key_ABAP,key_AMS,key_JAVA,key_ORACLE,key_CLOUD,key_EBS,key_DBA,key_PROXXI,key_C_HASH,key_OPERATIONS,key_PMO,key_ADM,key_MARKETING,key_TI,key_SALESFORCE,key_PROJETOS,key_DADOS,key_SECURITY,key_WEB,key_SCRUM,key_SERVICE,key_REACT_NATIVE_ANGULAR,key_NET,key_Analista,key_DEVOPS,key_PYTHON,key_C_PLUS_PLUS,key_COBOL,key_ANDROID,key_SQL,key_fiscal,key_software,nivel_vaga,tipo_escolha_do_candidato,tipo_CLT,tipo_PJ,tipo_cooperado,tipo_estagio,tipo_hunting,tipo_misto,score_ingles,score_espanhol,embedd_1,embedd_2,embedd_3,embedd_4,embedd_5,embedd_6,embedd_7,embedd_8,embedd_9,embedd_10,embedd_11,embedd_12,embedd_13,embedd_14,embedd_15,embedd_16,embedd_17,embedd_18,embedd_19,embedd_20,embedd_21,embedd_22,embedd_23,embedd_24,embedd_25,embedd_26,embedd_27,embedd_28,embedd_29,embedd_30,embedd_31,embedd_32,embedd_33,embedd_34,embedd_35,embedd_36,embedd_37,embedd_38,embedd_39,embedd_40,embedd_41,embedd_42,embedd_43,embedd_44,embedd_45,embedd_46,embedd_47,embedd_48,embedd_49,embedd_50,embedd_51,embedd_52,embedd_53,embedd_54,embedd_55,embedd_56,embedd_57,embedd_58,embedd_59,embedd_60,embedd_61,embedd_62,embedd_63,embedd_64,embedd_65,embedd_66,embedd_67,embedd_68,embedd_69,embedd_70,embedd_71,embedd_72,embedd_73,embedd_74,embedd_75,embedd_76,embedd_77,embedd_78,embedd_79,embedd_80,embedd_81,embedd_82,embedd_83,embedd_84,embedd_85,embedd_86,embedd_87,embedd_88,embedd_89,embedd_90,embedd_91,embedd_92,embedd_93,embedd_94,embedd_95,embedd_96,embedd_97,embedd_98,embedd_99,embedd_100,embedd_101,embedd_102,embedd_103,embedd_104,embedd_105,embedd_106,embedd_107,embedd_108,embedd_109,embedd_110,embedd_111,embedd_112,embedd_113,embedd_114,embedd_115,embedd_116,embedd_117,embedd_118,embedd_119,embedd_120,embedd_121,embedd_122,embedd_123,embedd_124,embedd_125,embedd_126,embedd_127,embedd_128,embedd_129,embedd_130,embedd_131,embedd_132,embedd_133,embedd_134,embedd_135,embedd_136,embedd_137,embedd_138,embedd_139,embedd_140,embedd_141,embedd_142,embedd_143,embedd_144,embedd_145,embedd_146,embedd_147,embedd_148,embedd_149,embedd_150,embedd_151,embedd_152,embedd_153,embedd_154,embedd_155,embedd_156,embedd_157,embedd_158,embedd_159,embedd_160,embedd_161,embedd_162,embedd_163,embedd_164,embedd_165,embedd_166,embedd_167,embedd_168,embedd_169,embedd_170,embedd_171,embedd_172,embedd_173,embedd_174,embedd_175,embedd_176,embedd_177,embedd_178,embedd_179,embedd_180,embedd_181,embedd_182,embedd_183,embedd_184,embedd_185,embedd_186,embedd_187,embedd_188,embedd_189,embedd_190,embedd_191,embedd_192,embedd_193,embedd_194,embedd_195,embedd_196,embedd_197,embedd_198,embedd_199,embedd_200,embedd_201,embedd_202,embedd_203,embedd_204,embedd_205,embedd_206,embedd_207,embedd_208,embedd_209,embedd_210,embedd_211,embedd_212,embedd_213,embedd_214,embedd_215,embedd_216,embedd_217,embedd_218,embedd_219,embedd_220,embedd_221,embedd_222,embedd_223,embedd_224,embedd_225,embedd_226,embedd_227,embedd_228,embedd_229,embedd_230,embedd_231,embedd_232,embedd_233,embedd_234,embedd_235,embedd_236,embedd_237,embedd_238,embedd_239,embedd_240,embedd_241,embedd_242,embedd_243,embedd_244,embedd_245,embedd_246,embedd_247,embedd_248,embedd_249,embedd_250,embedd_251,embedd_252,embedd_253,embedd_254,embedd_255,embedd_256,embedd_257,embedd_258,embedd_259,embedd_260,embedd_261,embedd_262,embedd_263,embedd_264,embedd_265,embedd_266,embedd_267,embedd_268,embedd_269,embedd_270,embedd_271,embedd_272,embedd_273,embedd_274,embedd_275,embedd_276,embedd_277,embedd_278,embedd_279,embedd_280,embedd_281,embedd_282,embedd_283,embedd_284,embedd_285,embedd_286,embedd_287,embedd_288,embedd_289,embedd_290,embedd_291,embedd_292,embedd_293,embedd_294,embedd_295,embedd_296,embedd_297,embedd_298,embedd_299,embedd_300,embedd_301,embedd_302,embedd_303,embedd_304,embedd_305,embedd_306,embedd_307,embedd_308,embedd_309,embedd_310,embedd_311,embedd_312,embedd_313,embedd_314,embedd_315,embedd_316,embedd_317,embedd_318,embedd_319,embedd_320,embedd_321,embedd_322,embedd_323,embedd_324,embedd_325,embedd_326,embedd_32

In [ ]:
pd.concat([df_unido_gerado,
               applicants_features_engineering(df_unido, 'applicants_cv_pt'),
               df_unido['target']], axis= 1
    )

In [183]:
df_embeddings = applicants_features_engineering(df_unido, 'applicants_cv_pt')

df_embeddings.head()

,embedd_1,embedd_2,embedd_3,embedd_4,embedd_5,embedd_6,embedd_7,embedd_8,embedd_9,embedd_10,embedd_11,embedd_12,embedd_13,embedd_14,embedd_15,embedd_16,embedd_17,embedd_18,embedd_19,embedd_20,embedd_21,embedd_22,embedd_23,embedd_24,embedd_25,embedd_26,embedd_27,embedd_28,embedd_29,embedd_30,embedd_31,embedd_32,embedd_33,embedd_34,embedd_35,embedd_36,embedd_37,embedd_38,embedd_39,embedd_40,embedd_41,embedd_42,embedd_43,embedd_44,embedd_45,embedd_46,embedd_47,embedd_48,embedd_49,embedd_50,embedd_51,embedd_52,embedd_53,embedd_54,embedd_55,embedd_56,embedd_57,embedd_58,embedd_59,embedd_60,embedd_61,embedd_62,embedd_63,embedd_64,embedd_65,embedd_66,embedd_67,embedd_68,embedd_69,embedd_70,embedd_71,embedd_72,embedd_73,embedd_74,embedd_75,embedd_76,embedd_77,embedd_78,embedd_79,embedd_80,embedd_81,embedd_82,embedd_83,embedd_84,embedd_85,embedd_86,embedd_87,embedd_88,embedd_89,embedd_90,embedd_91,embedd_92,embedd_93,embedd_94,embedd_95,embedd_96,embedd_97,embedd_98,embedd_99,embedd_100,embedd_101,embedd_102,embedd_103,embedd_104,embedd_105,embedd_106,embedd_107,embedd_108,embedd_109,embedd_110,embedd_111,embedd_112,embedd_113,embedd_114,embedd_115,embedd_116,embedd_117,embedd_118,embedd_119,embedd_120,embedd_121,embedd_122,embedd_123,embedd_124,embedd_125,embedd_126,embedd_127,embedd_128,embedd_129,embedd_130,embedd_131,embedd_132,embedd_133,embedd_134,embedd_135,embedd_136,embedd_137,embedd_138,embedd_139,embedd_140,embedd_141,embedd_142,embedd_143,embedd_144,embedd_145,embedd_146,embedd_147,embedd_148,embedd_149,embedd_150,embedd_151,embedd_152,embedd_153,embedd_154,embedd_155,embedd_156,embedd_157,embedd_158,embedd_159,embedd_160,embedd_161,embedd_162,embedd_163,embedd_164,embedd_165,embedd_166,embedd_167,embedd_168,embedd_169,embedd_170,embedd_171,embedd_172,embedd_173,embedd_174,embedd_175,embedd_176,embedd_177,embedd_178,embedd_179,embedd_180,embedd_181,embedd_182,embedd_183,embedd_184,embedd_185,embedd_186,embedd_187,embedd_188,embedd_189,embedd_190,embedd_191,embedd_192,embedd_193,embedd_194,embedd_195,embedd_196,embedd_197,embedd_198,embedd_199,embedd_200,embedd_201,embedd_202,embedd_203,embedd_204,embedd_205,embedd_206,embedd_207,embedd_208,embedd_209,embedd_210,embedd_211,embedd_212,embedd_213,embedd_214,embedd_215,embedd_216,embedd_217,embedd_218,embedd_219,embedd_220,embedd_221,embedd_222,embedd_223,embedd_224,embedd_225,embedd_226,embedd_227,embedd_228,embedd_229,embedd_230,embedd_231,embedd_232,embedd_233,embedd_234,embedd_235,embedd_236,embedd_237,embedd_238,embedd_239,embedd_240,embedd_241,embedd_242,embedd_243,embedd_244,embedd_245,embedd_246,embedd_247,embedd_248,embedd_249,embedd_250,embedd_251,embedd_252,embedd_253,embedd_254,embedd_255,embedd_256,embedd_257,embedd_258,embedd_259,embedd_260,embedd_261,embedd_262,embedd_263,embedd_264,embedd_265,embedd_266,embedd_267,embedd_268,embedd_269,embedd_270,embedd_271,embedd_272,embedd_273,embedd_274,embedd_275,embedd_276,embedd_277,embedd_278,embedd_279,embedd_280,embedd_281,embedd_282,embedd_283,embedd_284,embedd_285,embedd_286,embedd_287,embedd_288,embedd_289,embedd_290,embedd_291,embedd_292,embedd_293,embedd_294,embedd_295,embedd_296,embedd_297,embedd_298,embedd_299,embedd_300,embedd_301,embedd_302,embedd_303,embedd_304,embedd_305,embedd_306,embedd_307,embedd_308,embedd_309,embedd_310,embedd_311,embedd_312,embedd_313,embedd_314,embedd_315,embedd_316,embedd_317,embedd_318,embedd_319,embedd_320,embedd_321,embedd_322,embedd_323,embedd_324,embedd_325,embedd_326,embedd_327,embedd_328,embedd_329,embedd_330,embedd_331,embedd_332,embedd_333,embedd_334,embedd_335,embedd_336,embedd_337,embedd_338,embedd_339,embedd_340,embedd_341,embedd_342,embedd_343,embedd_344,embedd_345,embedd_346,embedd_347,embedd_348,embedd_349,embedd_350,embedd_351,embedd_352,embedd_353,embedd_354,embedd_355,embedd_356,embedd_357,embedd_358,embedd_359,embedd_360,embedd_361,embedd_362,embedd_363,embedd_364,embedd_365,embedd_366,embedd_367,embedd_368,embedd_369,embedd_370,embedd_371,embedd_372,embedd_373,embe

In [213]:
df_embeddings.reset_index(drop=True)

,embedd_1,embedd_2,embedd_3,embedd_4,embedd_5,embedd_6,embedd_7,embedd_8,embedd_9,embedd_10,embedd_11,embedd_12,embedd_13,embedd_14,embedd_15,embedd_16,embedd_17,embedd_18,embedd_19,embedd_20,embedd_21,embedd_22,embedd_23,embedd_24,embedd_25,embedd_26,embedd_27,embedd_28,embedd_29,embedd_30,embedd_31,embedd_32,embedd_33,embedd_34,embedd_35,embedd_36,embedd_37,embedd_38,embedd_39,embedd_40,embedd_41,embedd_42,embedd_43,embedd_44,embedd_45,embedd_46,embedd_47,embedd_48,embedd_49,embedd_50,embedd_51,embedd_52,embedd_53,embedd_54,embedd_55,embedd_56,embedd_57,embedd_58,embedd_59,embedd_60,embedd_61,embedd_62,embedd_63,embedd_64,embedd_65,embedd_66,embedd_67,embedd_68,embedd_69,embedd_70,embedd_71,embedd_72,embedd_73,embedd_74,embedd_75,embedd_76,embedd_77,embedd_78,embedd_79,embedd_80,embedd_81,embedd_82,embedd_83,embedd_84,embedd_85,embedd_86,embedd_87,embedd_88,embedd_89,embedd_90,embedd_91,embedd_92,embedd_93,embedd_94,embedd_95,embedd_96,embedd_97,embedd_98,embedd_99,embedd_100,embedd_101,embedd_102,embedd_103,embedd_104,embedd_105,embedd_106,embedd_107,embedd_108,embedd_109,embedd_110,embedd_111,embedd_112,embedd_113,embedd_114,embedd_115,embedd_116,embedd_117,embedd_118,embedd_119,embedd_120,embedd_121,embedd_122,embedd_123,embedd_124,embedd_125,embedd_126,embedd_127,embedd_128,embedd_129,embedd_130,embedd_131,embedd_132,embedd_133,embedd_134,embedd_135,embedd_136,embedd_137,embedd_138,embedd_139,embedd_140,embedd_141,embedd_142,embedd_143,embedd_144,embedd_145,embedd_146,embedd_147,embedd_148,embedd_149,embedd_150,embedd_151,embedd_152,embedd_153,embedd_154,embedd_155,embedd_156,embedd_157,embedd_158,embedd_159,embedd_160,embedd_161,embedd_162,embedd_163,embedd_164,embedd_165,embedd_166,embedd_167,embedd_168,embedd_169,embedd_170,embedd_171,embedd_172,embedd_173,embedd_174,embedd_175,embedd_176,embedd_177,embedd_178,embedd_179,embedd_180,embedd_181,embedd_182,embedd_183,embedd_184,embedd_185,embedd_186,embedd_187,embedd_188,embedd_189,embedd_190,embedd_191,embedd_192,embedd_193,embedd_194,embedd_195,embedd_196,embedd_197,embedd_198,embedd_199,embedd_200,embedd_201,embedd_202,embedd_203,embedd_204,embedd_205,embedd_206,embedd_207,embedd_208,embedd_209,embedd_210,embedd_211,embedd_212,embedd_213,embedd_214,embedd_215,embedd_216,embedd_217,embedd_218,embedd_219,embedd_220,embedd_221,embedd_222,embedd_223,embedd_224,embedd_225,embedd_226,embedd_227,embedd_228,embedd_229,embedd_230,embedd_231,embedd_232,embedd_233,embedd_234,embedd_235,embedd_236,embedd_237,embedd_238,embedd_239,embedd_240,embedd_241,embedd_242,embedd_243,embedd_244,embedd_245,embedd_246,embedd_247,embedd_248,embedd_249,embedd_250,embedd_251,embedd_252,embedd_253,embedd_254,embedd_255,embedd_256,embedd_257,embedd_258,embedd_259,embedd_260,embedd_261,embedd_262,embedd_263,embedd_264,embedd_265,embedd_266,embedd_267,embedd_268,embedd_269,embedd_270,embedd_271,embedd_272,embedd_273,embedd_274,embedd_275,embedd_276,embedd_277,embedd_278,embedd_279,embedd_280,embedd_281,embedd_282,embedd_283,embedd_284,embedd_285,embedd_286,embedd_287,embedd_288,embedd_289,embedd_290,embedd_291,embedd_292,embedd_293,embedd_294,embedd_295,embedd_296,embedd_297,embedd_298,embedd_299,embedd_300,embedd_301,embedd_302,embedd_303,embedd_304,embedd_305,embedd_306,embedd_307,embedd_308,embedd_309,embedd_310,embedd_311,embedd_312,embedd_313,embedd_314,embedd_315,embedd_316,embedd_317,embedd_318,embedd_319,embedd_320,embedd_321,embedd_322,embedd_323,embedd_324,embedd_325,embedd_326,embedd_327,embedd_328,embedd_329,embedd_330,embedd_331,embedd_332,embedd_333,embedd_334,embedd_335,embedd_336,embedd_337,embedd_338,embedd_339,embedd_340,embedd_341,embedd_342,embedd_343,embedd_344,embedd_345,embedd_346,embedd_347,embedd_348,embedd_349,embedd_350,embedd_351,embedd_352,embedd_353,embedd_354,embedd_355,embedd_356,embedd_357,embedd_358,embedd_359,embedd_360,embedd_361,embedd_362,embedd_363,embedd_364,embedd_365,embedd_366,embedd_367,embedd_368,embedd_369,embedd_370,embedd_371,embedd_372,embedd_373,embe

In [232]:
df_teste = CleanVagas(vagas.head(4)).clean_all().add_prefix("vagas_").rename(columns= {'vagas_titulo_vaga':'prospects_titulo'})

vagas_features_engineering(df_teste).create_all()

C:\Users\renan_dyoksic\AppData\Local\Temp\ipykernel_9804\3326073744.py:56: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  self.df[coluna] = self.df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)
C:\Users\renan_dyoksic\AppData\Local\Temp\ipykernel_9804\3326073744.py:56: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  self.df[coluna] = self.df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)
C:\Users\renan_dyoksic\AppData\Local\Temp\ipykernel_9804\3326073744.py:56: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  self.df[coluna] = self.df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)


,key_SAP,key_SD,key_MM,key_ABAP,key_AMS,key_JAVA,key_ORACLE,key_CLOUD,key_EBS,key_DBA,key_PROXXI,key_C_HASH,key_OPERATIONS,key_PMO,key_ADM,key_MARKETING,key_TI,key_SALESFORCE,key_PROJETOS,key_DADOS,key_SECURITY,key_WEB,key_SCRUM,key_SERVICE,key_REACT_NATIVE_ANGULAR,key_NET,key_Analista,key_DEVOPS,key_PYTHON,key_C_PLUS_PLUS,key_COBOL,key_ANDROID,key_SQL,key_fiscal,key_software,nivel_vaga,tipo_escolha_do_candidato,tipo_CLT,tipo_PJ,tipo_cooperado,tipo_estagio,tipo_hunting,tipo_misto,score_ingles,score_espanhol
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,4
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,4,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1


In [ ]:
def prepare_for_predict(vaga_informada: pd.DataFrame):
    
    df_embeddings = ???
    vaga_limpa = CleanVagas(vaga_informada)\
                    .clean_all()\
                    .add_prefix("vagas_")\
                    .rename(columns= {'vagas_titulo_vaga':'prospects_titulo'})
                    
    vaga_com_features = vagas_features_engineering(vaga_limpa).create_all()
    
    vaga_normalizada = normalize_dataframe(vaga_com_features)
    
    df_vaga_replicada = pd.concat([vaga_normalizada] * df_embeddings.shape[0], ignore_index= True)
    
    return pd.concat([df_vaga_replicada.reset_index(drop=True), df_embeddings.reset_index(drop=True)], axis= 1)

In [233]:
vaga_limpa = CleanVagas(vagas.head(1)).clean_all().add_prefix("vagas_").rename(columns= {'vagas_titulo_vaga':'prospects_titulo'})

vaga_com_features = vagas_features_engineering(vaga_limpa).create_all()

vaga_normalizada = normalize_dataframe(vaga_com_features)

vaga_normalizada


C:\Users\renan_dyoksic\AppData\Local\Temp\ipykernel_9804\3326073744.py:56: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  self.df[coluna] = self.df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)
C:\Users\renan_dyoksic\AppData\Local\Temp\ipykernel_9804\3326073744.py:56: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  self.df[coluna] = self.df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)
C:\Users\renan_dyoksic\AppData\Local\Temp\ipykernel_9804\3326073744.py:56: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  self.df[coluna] = self.df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)


,key_SAP,key_SD,key_MM,key_ABAP,key_AMS,key_JAVA,key_ORACLE,key_CLOUD,key_EBS,key_DBA,key_PROXXI,key_C_HASH,key_OPERATIONS,key_PMO,key_ADM,key_MARKETING,key_TI,key_SALESFORCE,key_PROJETOS,key_DADOS,key_SECURITY,key_WEB,key_SCRUM,key_SERVICE,key_REACT_NATIVE_ANGULAR,key_NET,key_Analista,key_DEVOPS,key_PYTHON,key_C_PLUS_PLUS,key_COBOL,key_ANDROID,key_SQL,key_fiscal,key_software,nivel_vaga,tipo_escolha_do_candidato,tipo_CLT,tipo_PJ,tipo_cooperado,tipo_estagio,tipo_hunting,tipo_misto,score_ingles,score_espanhol
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [240]:
df.index.to_list()

[0, 1, 2]

In [234]:
df_vaga_replicada = pd.concat([vaga_normalizada] * df_embeddings.shape[0], ignore_index= True)

pd.concat([df_vaga_replicada.reset_index(drop=True), df_embeddings.reset_index(drop=True)], axis= 1)

,key_SAP,key_SD,key_MM,key_ABAP,key_AMS,key_JAVA,key_ORACLE,key_CLOUD,key_EBS,key_DBA,key_PROXXI,key_C_HASH,key_OPERATIONS,key_PMO,key_ADM,key_MARKETING,key_TI,key_SALESFORCE,key_PROJETOS,key_DADOS,key_SECURITY,key_WEB,key_SCRUM,key_SERVICE,key_REACT_NATIVE_ANGULAR,key_NET,key_Analista,key_DEVOPS,key_PYTHON,key_C_PLUS_PLUS,key_COBOL,key_ANDROID,key_SQL,key_fiscal,key_software,nivel_vaga,tipo_escolha_do_candidato,tipo_CLT,tipo_PJ,tipo_cooperado,tipo_estagio,tipo_hunting,tipo_misto,score_ingles,score_espanhol,embedd_1,embedd_2,embedd_3,embedd_4,embedd_5,embedd_6,embedd_7,embedd_8,embedd_9,embedd_10,embedd_11,embedd_12,embedd_13,embedd_14,embedd_15,embedd_16,embedd_17,embedd_18,embedd_19,embedd_20,embedd_21,embedd_22,embedd_23,embedd_24,embedd_25,embedd_26,embedd_27,embedd_28,embedd_29,embedd_30,embedd_31,embedd_32,embedd_33,embedd_34,embedd_35,embedd_36,embedd_37,embedd_38,embedd_39,embedd_40,embedd_41,embedd_42,embedd_43,embedd_44,embedd_45,embedd_46,embedd_47,embedd_48,embedd_49,embedd_50,embedd_51,embedd_52,embedd_53,embedd_54,embedd_55,embedd_56,embedd_57,embedd_58,embedd_59,embedd_60,embedd_61,embedd_62,embedd_63,embedd_64,embedd_65,embedd_66,embedd_67,embedd_68,embedd_69,embedd_70,embedd_71,embedd_72,embedd_73,embedd_74,embedd_75,embedd_76,embedd_77,embedd_78,embedd_79,embedd_80,embedd_81,embedd_82,embedd_83,embedd_84,embedd_85,embedd_86,embedd_87,embedd_88,embedd_89,embedd_90,embedd_91,embedd_92,embedd_93,embedd_94,embedd_95,embedd_96,embedd_97,embedd_98,embedd_99,embedd_100,embedd_101,embedd_102,embedd_103,embedd_104,embedd_105,embedd_106,embedd_107,embedd_108,embedd_109,embedd_110,embedd_111,embedd_112,embedd_113,embedd_114,embedd_115,embedd_116,embedd_117,embedd_118,embedd_119,embedd_120,embedd_121,embedd_122,embedd_123,embedd_124,embedd_125,embedd_126,embedd_127,embedd_128,embedd_129,embedd_130,embedd_131,embedd_132,embedd_133,embedd_134,embedd_135,embedd_136,embedd_137,embedd_138,embedd_139,embedd_140,embedd_141,embedd_142,embedd_143,embedd_144,embedd_145,embedd_146,embedd_147,embedd_148,embedd_149,embedd_150,embedd_151,embedd_152,embedd_153,embedd_154,embedd_155,embedd_156,embedd_157,embedd_158,embedd_159,embedd_160,embedd_161,embedd_162,embedd_163,embedd_164,embedd_165,embedd_166,embedd_167,embedd_168,embedd_169,embedd_170,embedd_171,embedd_172,embedd_173,embedd_174,embedd_175,embedd_176,embedd_177,embedd_178,embedd_179,embedd_180,embedd_181,embedd_182,embedd_183,embedd_184,embedd_185,embedd_186,embedd_187,embedd_188,embedd_189,embedd_190,embedd_191,embedd_192,embedd_193,embedd_194,embedd_195,embedd_196,embedd_197,embedd_198,embedd_199,embedd_200,embedd_201,embedd_202,embedd_203,embedd_204,embedd_205,embedd_206,embedd_207,embedd_208,embedd_209,embedd_210,embedd_211,embedd_212,embedd_213,embedd_214,embedd_215,embedd_216,embedd_217,embedd_218,embedd_219,embedd_220,embedd_221,embedd_222,embedd_223,embedd_224,embedd_225,embedd_226,embedd_227,embedd_228,embedd_229,embedd_230,embedd_231,embedd_232,embedd_233,embedd_234,embedd_235,embedd_236,embedd_237,embedd_238,embedd_239,embedd_240,embedd_241,embedd_242,embedd_243,embedd_244,embedd_245,embedd_246,embedd_247,embedd_248,embedd_249,embedd_250,embedd_251,embedd_252,embedd_253,embedd_254,embedd_255,embedd_256,embedd_257,embedd_258,embedd_259,embedd_260,embedd_261,embedd_262,embedd_263,embedd_264,embedd_265,embedd_266,embedd_267,embedd_268,embedd_269,embedd_270,embedd_271,embedd_272,embedd_273,embedd_274,embedd_275,embedd_276,embedd_277,embedd_278,embedd_279,embedd_280,embedd_281,embedd_282,embedd_283,embedd_284,embedd_285,embedd_286,embedd_287,embedd_288,embedd_289,embedd_290,embedd_291,embedd_292,embedd_293,embedd_294,embedd_295,embedd_296,embedd_297,embedd_298,embedd_299,embedd_300,embedd_301,embedd_302,embedd_303,embedd_304,embedd_305,embedd_306,embedd_307,embedd_308,embedd_309,embedd_310,embedd_311,embedd_312,embedd_313,embedd_314,embedd_315,embedd_316,embedd_317,embedd_318,embedd_319,embedd_320,embedd_321,embedd_322,embedd_323,embedd_324,embedd_325,embedd_326,embedd_32